<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conectando o drive ao colab #

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# Treinar novo Modelo !

## 1. Baixando a CNN pré-treinada ##


In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

## 2. Carregando os arquivos do drive no colab ##

In [0]:
!unzip "/content/drive/My Drive/Colab/datasets/barcode3.zip"

## 3. Treinando o modelo ##

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=11, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

## 4. Avaliando os modelos gerados ##

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="/content/drive/My Drive/Colab/json/detection_config_barcode.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

**Avaliar modelo indivualmente.**

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

model ="/content/drive/My Drive/Colab/a/detection_model-ex-013--loss-0001.995.h5"
models= "barcode/models"

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
metrics = trainer.evaluateModel(model_path=models, json_path="/content/drive/My Drive/Colab/json/detection_config_barcode.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# Utilizar modelo já treinado.

## 1. Criando as pastas ##

In [92]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(dirs)
print("Número de corridas na pasta: ", file_count_corridas - 1)


newFolderCorrida = str(file_count_corridas)
folderCorrida = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida
print("..")
print("..")
print("..")
!mkdir $folderCorrida
print("Criando pasta 'Corrida_"+newFolderCorrida+"'")

folderTagsJson = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_json"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsJson
  print("Criando pasta '/tags_json'")
except:
  print("Já existe a pasta '/tags_json'")

folderTagsDetected = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_detected"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsDetected
  print("Criando pasta '/tags_detected'")
except:
  print("Já existe a pasta '/tags_detected'")

print("..")
print("..")
print("..")
try:
  !mkdir barcode-detected-objects
  print("Criando pasta '/barcode-detected-objects'")
except:
  print("Já existe a pasta '/barcode-detected-objects'")

Número de corridas na pasta:  0
..
..
..
Criando pasta 'Corrida_1'
..
..
..
Criando pasta '/tags_json'
..
..
..
Criando pasta '/tags_detected'
..
..
..
Criando pasta '/barcode-detected-objects'


## 2. Setando as variaveis ##


In [0]:
modelPath = "/content/drive/My Drive/Colab/models/barcode/detection_model-ex-007--loss-0003.066.h5"
JsonPath = "drive/My Drive/Colab/json/detection_config_barcode.json"
InputImage = "drive/My Drive/Colab/Imagens_Originais/img1.jpg"

## 3. Função de interseção sobre união e Função de eliminar quadrados sobrepostos.

In [0]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def boxArea(boxA, boxB):
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	return boxAArea/boxBArea

def removeDuplicatesLoop(detections):
	i=0
	k=0
	for i, detection in enumerate(detections):
		for k, detection in enumerate(detections):
					inter = bb_intersection_over_union(detections[i]["box_points"] , detections[k]["box_points"])
					if(inter > float(0.1) and inter < float(1)):
						if(boxArea(detections[i]["box_points"] , detections[k]["box_points"]) > 1):
							detections.pop(k)
						else:
							detections.pop(i)
							removeDuplicatesLoop(detections)

## 4. Treinando modelo e recortando os codigos de barras da imagem original ##


In [99]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image = InputImage, 
                                                           output_image_path="barcode-detected-OriginalBoxes.jpg",
                                                           minimum_percentage_probability=20)

print("Número       " , ": ", "Porcentagem ", "      :    ", "box_points")
print("----------------------------------------------------------------")
print("")

for e in detections:
  e['box_points'][0] = e['box_points'][0] - 50 ## x1
  e['box_points'][1] = e['box_points'][1] - 50 ## y1
  e['box_points'][2] = e['box_points'][2] + 50 ## x2
  e['box_points'][3] = e['box_points'][3] + 50 ## y2

removeDuplicatesLoop(detections)

for detection in detections:
  print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])

print("----------------------------------------------------------------")
print("Quantidade de código de barras detectados na imagem: ", len(detections))

Número        :  Porcentagem        :     box_points
----------------------------------------------------------------

barcode       :  57.23978877067566  :  [3328, 583, 3655, 820]
barcode       :  31.595543026924133  :  [2122, 946, 2530, 1326]
barcode       :  45.35054564476013  :  [2579, 955, 2932, 1278]
barcode       :  22.706034779548645  :  [3014, 1051, 3319, 1260]
barcode       :  31.375962495803833  :  [3339, 960, 3687, 1254]
barcode       :  56.83613419532776  :  [934, 1364, 1324, 1682]
barcode       :  49.271160364151  :  [1385, 1345, 1756, 1697]
barcode       :  41.60623550415039  :  [2568, 1333, 2931, 1654]
barcode       :  30.831488966941833  :  [2884, 1353, 3247, 1679]
barcode       :  52.50357389450073  :  [3339, 1320, 3689, 1664]
barcode       :  37.52295672893524  :  [72, 1517, 389, 1770]
barcode       :  40.25477468967438  :  [211, 1748, 560, 2074]
barcode       :  21.697892248630524  :  [1816, 1767, 2185, 2057]
barcode       :  40.55114686489105  :  [2125, 1734, 2514,

## 5. Identificando os Bounding Box e recortandando as imagens. 

In [100]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.segmentation
import cv2

Image_with_boxes = "barcode-detected-OriginalBoxes.jpg"

img = skimage.io.imread(InputImage)
bbox = []

for e in detections:
    bbox.append(e['box_points'])

for i, (x1,y1,x2,y2) in enumerate(bbox):

    if i in {0,1,2,3,4,5,6,7,8,9}:
      i = '00' + str(i)
    
    if(int(i) > 9 and int(i) < 100):
      i = '0' + str(i)

    img2 = cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 0)
    out = img[y1:y2, x1:x2]

    cv2.imwrite('barcode-detected-objects/barcode-00'+str(i)+'.jpg', out)

cv2.imwrite('barcode-detected-New.jpg', img)

print("Boxes recortados com sucesso!")
print("..")
print("..")
print("..")
print("Imagem = 'barcode-detected-New.jpg' gerada.")

Boxes recortados com sucesso!
..
..
..
Imagem = 'barcode-detected-New.jpg' gerada.


**Visualizando nova a imagem gerada a partir das novas Bounding Boxes.**

In [0]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)

## 6. Movendo as pastas para o diretório correto. ##


In [49]:
!mv /content/barcode-detected-OriginalBoxes.jpg $folderCorrida
!mv /content/barcode-detected-New.jpg $folderCorrida
!mv /content/barcode-detected-objects $folderCorrida

barcode = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "/content/drive/My\ Drive/Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_images"
!mv  $barcode $images

print("Pastas movidas para o drive com sucesso!")

Pastas movidas para o drive com sucesso!
